In [1]:
import os
os.chdir("../../")

import numpy as np
import pandas as pd

from scripts.python.tsa.mtsmodel import *
from scripts.python.tsa.ts_eval import *

In [3]:
test = RatioPipe(country="palau", data=None, 
                 var_name=["total", "seats_arrivals_intl"])
test.read_and_merge()
test.transform()
test.fit()

,date,japan,south_korea,taiwan,china,usa/canada,europe,others,total,palau_flights,palau_hotel,palau_travel,year,month,stringency_index,covid
0,2007-06-01,856.0,1291.0,3245.0,86.0,669.0,99.0,463.0,6709.0,1.068426,8.334537,0.320004,0.0,0.0,0.0,0
1,2007-07-01,2119.0,1366.0,3269.0,33.0,653.0,144.0,437.0,8021.0,0.000000,9.667343,0.952482,0.0,0.0,0.0,0
2,2007-08-01,3476.0,1354.0,3046.0,46.0,580.0,256.0,438.0,9196.0,0.704914,6.906282,0.453967,0.0,0.0,0.0,0
3,2007-09-01,3022.0,910.0,2497.0,61.0,559.0,145.0,401.0,7595.0,0.798284,3.591027,0.000000,0.0,0.0,0.0,0
4,2007-10-01,1807.0,1082.0,2298.0,49.0,774.0,390.0,395.0,6795.0,0.431326,4.111283,0.000000,0.0,0.0,0.0,0


,date,seats_arrivals_total,seats_arrivals_intl,number_of_flights_total,number_of_flights_intl,japan,south_korea,taiwan,china,usa/canada,europe,others,total,palau_flights,palau_hotel,palau_travel,year,month,stringency_index,covid
0,2019-01-01,13048,13048,75,75,1953.0,1169.0,919.0,2072.0,626.0,310.0,280.0,7329.0,0.493228,2.559978,0.194222,0.0,0.0,0.0,0
1,2019-02-01,13281,13281,74,74,2055.0,1035.0,1092.0,4059.0,702.0,438.0,371.0,9752.0,0.310671,2.841414,0.282245,0.0,0.0,0.0,0
2,2019-03-01,12870,12870,73,73,2434.0,1090.0,1190.0,2549.0,826.0,483.0,461.0,9033.0,0.386389,2.921431,0.286529,0.0,0.0,0.0,0
3,2019-04-01,10806,10806,64,64,1756.0,808.0,1099.0,3182.0,777.0,368.0,415.0,8405.0,0.293529,2.981907,0.338945,0.0,0.0,0.0,0
4,2019-05-01,11472,11472,67,67,1288.0,1095.0,1393.0,2489.0,607.0,185.0,421.0,7478.0,0.306351,3.537987,0.168664,0.0,0.0,0.0,0


Abnormal value produced.
                            WLS Regression Results                            
Dep. Variable:                  ratio   R-squared:                       0.762
Model:                            WLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     16.95
Date:                Tue, 02 May 2023   Prob (F-statistic):           8.36e-10
Time:                        17:06:46   Log-Likelihood:                 34.253
No. Observations:                  45   AIC:                            -52.51
Df Residuals:                      37   BIC:                            -38.05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Int